In [543]:
import random
import numpy as np
import copy
class Node:
    def __init__(self,speed,cpu,id,BlkID,block):
        self.cpu_type = cpu
        self.speed = speed
        self.id = id
     
        self.txPool = set()
        self.bitcoinTree = {}
        self.bitcoinTree[BlkID] = block
        self.currentHead = BlkID
class Event:
    def __init__(self,event_type,time,peer_id,parameters,fromNode,newBlock,tx):
        self.type = event_type
        self.time = time
        self.peer_id = peer_id
        self.parameters = parameters
        self.newBlock = newBlock
        self.fromNode = fromNode
        self.tx = tx
    
    def __lt__(self,other):
        return self.time < other.time


class Block:
    def __init__(self,type,BlkID,prevBlkID, createdAt, txList,amount,timeArrival):
        self.type = type
        self.BlkID = BlkID
        self.prevBlkID = prevBlkID
        self.createdAt = createdAt
        self.txList = txList
        self.amountDic = amount
        self.timeArrival = timeArrival

In [621]:
####### Initialization Functions

import random
import string
import secrets


def initialize(num_peers,z0,z1):
    
    peers =[]
    amount={}
    for peer_id in range(num_peers):
        amount[peer_id] = np.random.randint(100,1000)
    
    

    genesisBlock = Block('genesis',genUID(),None,0,None,amount,0)
    for peer_id in range(num_peers):
        
        temp_z0 = np.random.uniform(0,1)
        temp_z1 = np.random.uniform(0,1)
        speed =  "slow" if temp_z0 < z0 else  "fast" 
        cpu =  "low CPU" if temp_z1 < z1  else "high CPU"
        
        peers.append(Node(speed,cpu,peer_id,genesisBlock.BlkID,copy.deepcopy(genesisBlock)))
        
    return peers

def check_connected(num_peers,cn):
    visited=[0 for i in range(num_peers)]
    index=0
    visited = dfs(index,cn,visited)
    return all(visited)
        

def dfs(index,connected,visited):
    visited[index] = 1
    for peer in connected[index]:
        if visited[peer]==0:
            dfs(peer,connected,visited)
    return visited    
    

def make_connections(num_peers,startRange,stopRange):
    
    pij = {}
    connected = [[] for i in range(num_peers)]
    for i in range(0,num_peers):
        
        edges = np.random.randint(startRange,stopRange+1)
        if len(connected[i]) in range(startRange,stopRange+1):
            continue
            
        while (len(connected[i]) < edges ):
            node_id = random.choice([index for index in range(num_peers) if index not in connected[i]])
            if node_id!= i and len(connected[node_id]) < stopRange:
                    ij = np.random.uniform(1e-2,0.5)
                    pij[str(i)+"-"+str(node_id)] = ij
                    pij[str(node_id)+"-"+str(i)] = ij
                    connected[i].append(node_id)
                    connected[node_id].append(i)         
            
    if check_connected(num_peers,connected):
        return (connected,pij)
    else:
        make_connections(num_peers,startRange,stopRange)

In [614]:
######### Transaction Functions

def generate_transaction(peer_id,parameters,aTime,eventQueue):
    meanTime = 2
    peers = parameters['peers']
    num_peers = parameters['num_peers']
    network_topology = parameters['network_topology']
    pij = parameters['pij']
    
    txid= ''.join(secrets.choice(string.ascii_uppercase + string.ascii_lowercase) for i in range(7))
    idy = None
    while(1) :
        idy = np.random.randint(0,num_peers)
        if idy!= peer_id:
            break
    
    amount = np.random.randint(1,101)
    idx = peer_id
    # peer[peer_id].credit -= amount
    
    tx = txid+": "+str(idx)+" pays "+str(idy)+" "+str(amount)+" coins"
    mlen = 8e+3

    
    peers[peer_id].txPool.add(tx)
    
    
    for adj_node in network_topology[peer_id]:
        
        # print(f'GenTx fromNode {peer_id} adj {adj_node} {tx}')
        
        cij = 1e+8 if ( peers[peer_id].speed == 'fast' and peers[adj_node].speed == 'fast') else 5e+6
        plight = pij[str(idx)+"-"+str(adj_node)]
        dij = np.random.exponential(scale= (96e+3/cij))
        newTime = aTime + plight + (mlen/cij) + dij
        fromNode = peer_id
        newEvent = Event("receiveTx",newTime, adj_node, parameters,fromNode , None,tx )
        eventQueue.put(newEvent)
        
    arrivalTime = aTime + np.random.exponential(scale=meanTime)
    eventQueue.put(Event("generate_transaction",arrivalTime,peer_id,parameters,None,None,None))
        
        

def receiveTx(peer_id,parameters,aTime,fromNode,tx,eventQueue):
    
    peer = parameters['peers']
    num_peers = parameters['num_peers']
    network_topology = parameters['network_topology']
    pij = parameters['pij']    
    
    if tx in peer[peer_id].txPool:
        return 
    
    peer[peer_id].txPool.add(tx)
    mlen = 8e+3
    
    for adj_node in network_topology[peer_id]:
        
        if adj_node!= fromNode:
            # print(f'ReceiveTx fromNode {fromNode} adj {adj_node} {tx}')
            cij = 1e+8 if ( peer[peer_id].speed == 'fast' and peer[adj_node].speed == 'fast') else 5e+6
            plight = pij[str(peer_id)+"-"+str(adj_node)]
            dij = np.random.exponential(scale= (96e+3/cij))
            newTime = aTime + plight + (mlen/cij) + dij
            newFromNode = peer_id
            newEvent = Event("receiveTx",newTime, adj_node, parameters,newFromNode,None,tx )
            eventQueue.put(newEvent)


In [650]:
### Block Functions

def compute_hashPower(peer):
    hpw=0
    numPeers =0
    for node in peer:
        hpw += 10 if node.cpu_type == "high CPU" else 1
        numPeers +=1
        
    return hpw/numPeers
    
def genUID():
    return ''.join(secrets.choice(string.ascii_uppercase + string.ascii_lowercase) for i in range(8))
        

def checkValidTx(tmpTx,amountDic,peer_id,parameters):
    
    peers = parameters['peers']
    currentBlock = peers[peer_id].bitcoinTree[peers[peer_id].currentHead]
    # print(currentBlock.type)
    # print("peer_id",peer_id)
    # print("prevBlock",currentBlock.prevBlkID)
    while ( currentBlock.type != 'genesis' ):
        txList = currentBlock.txList
        
        if tmpTx in txList:
            return False
        currentBlock = peers[peer_id].bitcoinTree[currentBlock.prevBlkID]
        
        
    
    tmp = tmpTx.split()
    idx = int(tmp[1])
    if tmp[2] == 'mines':
        amountDic[idx] +=50
        return True
    
    idy = int(tmp[3])
    coins = int(tmp[4])
    if amountDic[idx] >= coins:
        amountDic[idx] -=coins
        return True

    return False

            
def mineBlock(peer_id,parameters,aTime,eventQueue):
    
    peers = parameters['peers']
    num_peers = parameters['num_peers']
    txPool = peers[peer_id].txPool
    amountDic = peers[peer_id].bitcoinTree[peers[peer_id].currentHead].amountDic.copy()
    txList = set()
    hashpw = parameters['hashpw']
    hpw = 10*hashpw if peers[peer_id].cpu_type == 'high CPU' else hashpw
    numPops =0
    txid= ''.join(secrets.choice(string.ascii_uppercase + string.ascii_lowercase) for i in range(7)) 
    
    tx = txid+": "+str(peer_id)+" mines 50 coins"
    
    txList.add(tx)
    
    while( len(txPool) > 0 and numPops < 999):
        tmpTx = txPool.pop()
        ### check validTxs 
        if checkValidTx(tmpTx,amountDic, peer_id,parameters):
            txList.add(tmpTx)
            numPops +=1
    
    amountDic[peer_id] +=50
    
    avgTime = 200
    time_Tk = np.random.exponential(scale=avgTime/hpw)
    newTime = aTime + time_Tk
    newBlock = Block('valid',genUID(),peers[peer_id].currentHead,newTime,txList,amountDic,newTime)
    newEvent = Event("genBlock",newTime,peer_id, parameters,None,newBlock,None)
    eventQueue.put(newEvent)
    
    
    
def genBlock(peer_id,parameters,aTime,newBlock,eventQueue):
    
    peers = parameters['peers']
    
    if peers[peer_id].currentHead == newBlock.prevBlkID :
        peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)
        peers[peer_id].currentHead = newBlock.BlkID     
        
        fromNode= peer_id 
       
        newEvent = Event("broadcastBlock",aTime,peer_id,parameters,fromNode,newBlock,None)
        eventQueue.put(newEvent)
        
        
    else:
        
        while ( len(newBlock.txList) > 0 ):
            peers[peer_id].txPool.add(newBlock.txList.pop())
            
        eventQueue.put(Event("mineBlock",aTime,peer_id,parameters,None,None,None))
    
  
        
    
    
def broadcastBlock(peer_id,parameters,aTime,fromNode,newBlock,eventQueue):
    
    peers = parameters['peers']
    network_topology = parameters['network_topology']
    pij = parameters['pij']

    # if newBlock.BlkID in peers[peer_id].bitcoinTree.keys():
    #     return
    
    
    mlen = 8e+6 ## assumed blockSize = 1MB
    
    for adj_node in network_topology[peer_id]:
        
        if adj_node!= fromNode:      
            cij = 1e+8 if ( peers[peer_id].speed == 'fast' and peers[adj_node].speed == 'fast') else 5e+6
            plight = pij[str(peer_id)+"-"+str(adj_node)]
            dij = np.random.exponential(scale= (96e+3/cij))
            newTime = aTime + plight + (mlen/cij) + dij
            newFromNode = peer_id
            newEvent = Event("receiveBlock",newTime, adj_node, parameters,newFromNode,newBlock,None )
            eventQueue.put(newEvent)
       
    newEvent = Event('mineBlock',aTime,peer_id,parameters,None,None,None)
    eventQueue.put(newEvent)  

def getValidBlockAndDepth(peer_id,parameters,newBlock):
   
    peers = parameters['peers']
    bitcoinTree = peers[peer_id].bitcoinTree
    tempBlock = bitcoinTree[newBlock.BlkID]
    depth = 0
    while( tempBlock.type != 'valid' and tempBlock.type !='genesis'):
        depth +=1
        tempBlock = bitcoinTree[tempBlock.prevBlkID]
   
    return (tempBlock , depth)
    
    
def getDepth(peer_id,parameters,block):
    

    peers = parameters['peers']
    bitcoinTree = peers[peer_id].bitcoinTree
    currentBlock = bitcoinTree[peers[peer_id].currentHead]
    
    tempBlock = bitcoinTree[currentBlock.BlkID]
    depth = 0

    ##### may be not found case
    
    while (tempBlock.BlkID != block.BlkID):
        depth +=1
        tempBlock = bitcoinTree[tempBlock.prevBlkID]
    
    return depth
    
    
def makeOrphanAndReleaseTx(peer_id,parameters,validBlock):
    peers = parameters['peers']
    bitcoinTree = peers[peer_id].bitcoinTree
    tempBlock = bitcoinTree[peers[peer_id].currentHead]
    
    while tempBlock.BlkID != validBlock.BlkID :
        tempBlock.type = "orphan"
        txList = tempBlock.txList.copy()
        while len(txList) > 0 :
            tmpTx = txList.pop()
            tmp = tmpTx.split()
            if tmp[2] != "mines":
                peers[peer_id].txPool.add(tmpTx)
            
        tempBlock = bitcoinTree[tempBlock.prevBlkID]
   
 


def makeValidAndProcessTx(peer_id,parameters,validBlock):
    
    peers = parameters['peers']
    bitcoinTree = peers[peer_id].bitcoinTree
    tempBlock = bitcoinTree[peers[peer_id].currentHead]
    
    while tempBlock.BlkID != validBlock.BlkID :
        tempBlock.type = 'valid'
        txList = tempBlock.txList.copy()
        while len(txList) > 0:
            tmpTx = txList.pop()
            tmp = tmpTx.split()
            if tmp[2] != 'mines':
                try:
                    peers[peer_id].txPool.remove(tmpTx)
                except:
                    continue
            
        tempBlock = bitcoinTree[tempBlock.prevBlkID]
        

    


def checkValidBlock(peer_id,parameters,block):
    
    peers = parameters['peers']
    
    
    if block.prevBlkID not in peers[peer_id].bitcoinTree.keys():
        # print("Earlier not Came")
        return False
    
    amountDic = peers[peer_id].bitcoinTree[block.prevBlkID].amountDic.copy()
    txList = block.txList.copy()
    
    for tx in txList:
        if checkValidTx(tx,amountDic,peer_id,parameters)==False :
            # print("Not Valid Tx ",False)
            return False
        
    if amountDic == block.amountDic :
        # print("amount Matched and Valid tx",True)
        return True
    else:
        print("amount NOt Matched",False)
        return False
    
    
def receiveBlock(peer_id,parameters,aTime,fromNode,newBlock,eventQueue):
    
    peers = parameters['peers']
    
    # for a in peers:
    #     backTrack(a.id,parameters,peers[a.id].currentHead)
    
    
    if checkValidBlock(peer_id,parameters,newBlock)==False:
        return
    
    newBlock.timeArrival = aTime  ###check here
    
    if peers[peer_id].currentHead == newBlock.prevBlkID :
        
        #### check valid Tx's
        peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)
        peers[peer_id].currentHead = newBlock.BlkID
  
        #### remove Tx from txPool
        txList = newBlock.txList.copy()
            
        while len(txList) > 0:
            try:
                peers[peer_id].txPool.remove(txList.pop())
            except:
                continue

        newEvent = Event("broadcastBlock",aTime,peer_id,parameters,peer_id,newBlock,None)
        eventQueue.put(newEvent)
        newEvent = Event("mineBlock",aTime,peer_id,parameters,None,None,None)
        eventQueue.put(newEvent)

    else :
        
        if newBlock.prevBlkID not in  peers[peer_id].bitcoinTree.keys():
            return
        
        if peers[peer_id].bitcoinTree[newBlock.prevBlkID].type == 'orphan':
            
            newBlock.type = "orphan"
            peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)
            
            # print("BACKTRACK after adding orphan above orphan")
            # backTrack(peer_id,parameters,newBlock.BlkID)
            
            (validBlock, depth ) = getValidBlockAndDepth(peer_id,parameters,newBlock)
            
            # print(f"Got validBlock {validBlock.BlkID} at depth {depth}")
                
            
            
#             print("BACKTRACK from current Head")
#             backTrack(peer_id,parameters,peers[peer_id].currentHead)
            
            validDepth = getDepth(peer_id,parameters,validBlock)
            # print(f"Got validBlock {validBlock.BlkID} at depth {validDepth}")
            

            if validDepth < depth :
                # print("ValidDepth is less so interchange")
                # print("original from currentHead")
                # tmpCurrentHead = peers[peer_id].currentHead
                # backTrack(peer_id,parameters,peers[peer_id].currentHead)
                # print("original from newBlock")
                # backTrack(peer_id,parameters,newBlock.BlkID)
                
                makeOrphanAndReleaseTx(peer_id,parameters,validBlock)
                peers[peer_id].currentHead = newBlock.BlkID
                peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)
                makeValidAndProcessTx(peer_id,parameters,validBlock)
                
                
                # print("original from currentHead new")
                # backTrack(peer_id,parameters,peers[peer_id].currentHead)
                # print("original from old CurrentHead")
                # backTrack(peer_id,parameters,tmpCurrentHead)
              
                
                newEvent = Event("mineBlock",aTime,peer_id,parameters,None,None,None)
                eventQueue.put(newEvent)
                
            
            
            else:
                newBlock.type = "orphan"
                peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)

        
        else:
            
            newBlock.type = "orphan"
            peers[peer_id].bitcoinTree[newBlock.BlkID] = copy.deepcopy(newBlock)

            
            

def backTrack(peer_id,parameters,blkID):
    print(f"Backtrack for peer {peer_id}")
    peers = parameters['peers']
    bitcoinTree = peers[peer_id].bitcoinTree
    
    tmp = bitcoinTree[blkID]
    
    while(tmp.type !='genesis'):
        print(tmp.type,tmp.BlkID," pointing to ",tmp.prevBlkID)
        tmp = bitcoinTree[tmp.prevBlkID]
    
    print(tmp.type,tmp.BlkID," pointing to ",tmp.prevBlkID)
    
    print()
    print()
    
    
    
        
    

In [657]:
from queue import PriorityQueue
# np.random.seed(15)
# random.seed(5)
z0 = np.random.uniform(0,1)
z1 = np.random.uniform(0,1)
num_peers = np.random.randint(50,124)
startRange = 4
stopRange = 8
peers = initialize(num_peers,z0,z1)
(network_topology,pij) = make_connections(num_peers,startRange,stopRange)
l=[]
for a in network_topology:
    l.append(len(a))
    print(a)

l = np.array(l)
print(l.max(),l.min())

meanTime = 5
hashpw = compute_hashPower(peers)
parameters = {
    "num_peers" : num_peers,
    "peers" : peers,
    "network_topology" : network_topology,
    "pij" : pij ,
    "hashpw" : hashpw
    }


[1, 9, 64, 19, 81, 75, 84, 63]
[0, 8, 6, 102, 99, 86, 37]
[92, 76, 39, 22, 85, 28, 81, 34]
[13, 31, 11, 87, 21, 33, 54, 82]
[103, 87, 88, 81, 17]
[69, 90, 93, 102, 9, 11, 63, 41]
[1, 80, 94, 8, 7, 16, 20, 29]
[6, 40, 80, 54, 35]
[1, 6, 28, 75, 23, 57, 101]
[0, 5, 73, 27, 26, 69, 14, 22]
[90, 68, 49, 13, 92, 40, 19, 62]
[3, 5, 27, 100, 89, 23, 37, 59]
[75, 63, 25, 24, 66, 60, 19, 42]
[3, 10, 59, 65, 35, 84, 45, 16]
[35, 67, 44, 54, 9, 49, 60]
[44, 86, 72, 77, 96, 18, 30, 49]
[34, 66, 23, 6, 13, 40, 19, 53]
[69, 71, 79, 4, 51, 103, 90, 26]
[51, 81, 40, 57, 19, 20, 15, 33]
[0, 10, 12, 16, 18]
[18, 6, 83, 75, 42, 51, 32]
[57, 3, 99, 27, 42, 78]
[2, 77, 53, 9, 55, 33, 34, 35]
[16, 8, 11, 39, 40, 32, 33, 94]
[12, 32, 78, 67, 95]
[12, 103, 30, 65, 52, 68]
[9, 86, 41, 17, 53, 79, 66, 61]
[9, 11, 21, 72, 29]
[2, 8, 65, 77, 84, 98, 80, 60]
[6, 88, 97, 27, 36, 48]
[25, 89, 96, 64, 72, 39, 15, 34]
[3, 52, 100, 76, 33, 58]
[20, 23, 24, 74, 44, 48]
[23, 22, 31, 3, 80, 18, 78, 54]
[2, 16, 30, 22, 100

In [658]:


eventQueue = PriorityQueue()
for peer_id in range(num_peers):

    arrivalTime=0
    # for transaction in range(1):
    arrivalTime =np.random.exponential(scale=meanTime)
    eventQueue.put(Event("generate_transaction",arrivalTime,peer_id,parameters,None,None,None))
    
    eventQueue.put(Event("mineBlock",arrivalTime + np.random.uniform(2,3),peer_id,parameters,None,None,None))

c=0  
import os
try:
    os.remove("file.txt")
except:
    pass
f = open("file.txt", "a")

while not eventQueue.empty():
    next_schedule = eventQueue.get()
    c+=1
    
    if ( c > 1111010 ):
        break
    
    
    
    
    # print(next_schedule.time,next_schedule.peer_id,next_schedule.type,)
    if next_schedule.type == "generate_transaction" :
        generate_transaction(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,eventQueue)
        f.write(str(next_schedule.time)+" "+str(next_schedule.peer_id)+" "+next_schedule.type+"\n")
       
        
    if next_schedule.type == 'receiveTx':
        receiveTx(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,next_schedule.fromNode,next_schedule.tx,eventQueue)
        f.write(str(next_schedule.time)+" from "+str(next_schedule.fromNode)+" to "+str(next_schedule.peer_id)+" "+next_schedule.tx+"\n")
       
        
        
    if next_schedule.type == 'mineBlock':
        mineBlock(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,eventQueue)
        f.write(str(next_schedule.time)+" "+str(next_schedule.peer_id)+" "+next_schedule.type+"\n")
        
        
        
    if next_schedule.type == 'genBlock':
        genBlock(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,next_schedule.newBlock,eventQueue)
        f.write(str(next_schedule.time)+" "+str(next_schedule.peer_id)+" "+next_schedule.type+"\n")
    
    if next_schedule.type == 'broadcastBlock':
        broadcastBlock(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,next_schedule.fromNode,next_schedule.newBlock,eventQueue)
        f.write(str(next_schedule.time)+" "+str(next_schedule.peer_id)+" "+next_schedule.type+"\n")
        
        
    if next_schedule.type == 'receiveBlock':
        receiveBlock(next_schedule.peer_id,next_schedule.parameters,next_schedule.time,next_schedule.fromNode,next_schedule.newBlock,eventQueue)
        f.write(str(next_schedule.time)+" from "+str(next_schedule.fromNode)+" to "+str(next_schedule.peer_id)+" "+next_schedule.type+"\n")
f.close()

In [659]:
for a in peers:
    valid =0
    invalid=0
    for k in a.bitcoinTree:
       
        if a.bitcoinTree[k].type == 'valid':
            valid +=1
        else:
            invalid+=1
    print(f' peer_id {a.id} has {valid} valid blocks and {invalid} invalid blocks')
            
        

 peer_id 0 has 5 valid blocks and 6 invalid blocks
 peer_id 1 has 5 valid blocks and 2 invalid blocks
 peer_id 2 has 4 valid blocks and 6 invalid blocks
 peer_id 3 has 4 valid blocks and 6 invalid blocks
 peer_id 4 has 4 valid blocks and 5 invalid blocks
 peer_id 5 has 4 valid blocks and 8 invalid blocks
 peer_id 6 has 4 valid blocks and 6 invalid blocks
 peer_id 7 has 7 valid blocks and 6 invalid blocks
 peer_id 8 has 4 valid blocks and 4 invalid blocks
 peer_id 9 has 3 valid blocks and 6 invalid blocks
 peer_id 10 has 6 valid blocks and 7 invalid blocks
 peer_id 11 has 3 valid blocks and 6 invalid blocks
 peer_id 12 has 6 valid blocks and 5 invalid blocks
 peer_id 13 has 5 valid blocks and 10 invalid blocks
 peer_id 14 has 5 valid blocks and 3 invalid blocks
 peer_id 15 has 6 valid blocks and 7 invalid blocks
 peer_id 16 has 4 valid blocks and 7 invalid blocks
 peer_id 17 has 6 valid blocks and 5 invalid blocks
 peer_id 18 has 6 valid blocks and 7 invalid blocks
 peer_id 19 has 4 val

In [660]:
for a in peers:
    print(a.id)
    for b in  a.bitcoinTree.keys():
        print(a.bitcoinTree[b].timeArrival , a.bitcoinTree[b].BlkID)
    

0
0 EedGRUcN
4.1009390274489865 zohDwwOA
4.381979274316774 EpKIlOlr
5.2022260678846175 RToCNAdj
5.385473222739698 rhotslXZ
5.69465188420988 ceBAimLP
6.8405003721805615 xDtaNSOG
6.998358076932414 HrKTYkPW
7.621355717667194 csrGCGDJ
10.029697909792063 vcQdRfNR
11.435775834840047 iBkLHmMH
1
0 EedGRUcN
4.187207648768641 zohDwwOA
8.033244837975342 EpKIlOlr
5.089945495830551 RToCNAdj
6.071766355837717 UtroclXp
8.763330339565108 opfPLGtD
9.596205663603547 swAQUOjO
2
0 EedGRUcN
3.7183428075615654 zohDwwOA
4.744090804807951 rhotslXZ
4.94477931658696 qQLjKhQE
5.1245292617240175 XPnIVkmV
5.587915703759989 ceBAimLP
5.772288124538422 xDtaNSOG
6.549826719722567 csrGCGDJ
6.834085972833709 HrKTYkPW
10.060010858858345 aHSBuVyK
3
0 EedGRUcN
3.9398086992678967 zohDwwOA
4.69572964523558 XPnIVkmV
5.282390983093371 rhotslXZ
5.059627828327799 qQLjKhQE
5.217620976893612 EpKIlOlr
5.300093455464577 ceBAimLP
6.460802329096155 EizycqEG
6.725384662621821 HrKTYkPW
8.572985840117903 fuEFkcXF
4
0 EedGRUcN
3.928302829